# Pretrained Model, regression (target MEM-Score), Elastic Net


In [1]:
import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github') 

Mounted at /content/drive


In [2]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files
from src.preprocessing.data_preparation import prepare_data

In [4]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'aggregation' 

df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = preprocessing_type, statistic = 'greater_zero')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [5]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,1.0,0.565417,0.438914,0.512821,0.490598,0.590909,0.54931,0.514793,0.542735,0.903114,0.923351,0.951961,0.671123,0.806561,0.826923,0.808007,0.964187,0.955556,0.913223,0.657343,0.783217,0.843434,0.962222,0.759091,0.714103,0.903846,0.875,0.822314,0.512238,0.694056,0.727273,0.718935,0.652367,0.662393,0.89645,0.931624,0.91821
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,0.607495,0.460784,0.515929,0.500855,0.536131,0.434911,0.519724,0.59188,0.880623,0.901961,0.82451,0.522727,0.719457,0.297511,0.55719,0.96786,0.934343,0.884298,0.722611,0.470862,0.645623,0.864444,0.721212,0.507692,0.5,0.537963,0.900826,0.736014,0.624126,0.585859,0.85503,0.387574,0.657051,0.718935,0.647436,0.802469
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,3.0,0.628534,0.503771,0.54157,0.578632,0.558275,0.542406,0.610454,0.541311,0.799308,0.672906,0.647059,0.46123,0.649321,0.496606,0.598039,0.947658,0.835354,0.823691,0.600233,0.543124,0.479798,0.848889,0.786364,0.585897,0.710256,0.450926,0.838843,0.501748,0.601399,0.464646,0.745562,0.618343,0.632479,0.689349,0.599359,0.712963
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,4.0,0.78501,0.536199,0.302253,0.347009,0.576923,0.548323,0.588757,0.666667,0.930796,0.523173,0.34902,0.589572,0.593891,0.331448,0.601307,0.719927,0.629293,0.698347,0.452214,0.47669,0.380471,0.675556,0.543939,0.382051,0.642308,0.441667,0.72314,0.417832,0.536713,0.487374,0.653846,0.461538,0.57265,0.727811,0.642094,0.743827
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,5.0,0.682446,0.585219,0.686869,0.624786,0.648019,0.523669,0.623274,0.678775,0.745675,0.468806,0.657843,0.59492,0.536199,0.744344,0.762255,0.831956,0.729293,0.753444,0.460373,0.655012,0.679293,0.837778,0.734848,0.484615,0.902564,0.750926,0.760331,0.43007,0.746503,0.703283,0.659763,0.491124,0.667735,0.91716,0.885684,0.950617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,906,f816d269a,16,84,2017-11-21,2,3,18,-2.388095,0.0,906.0,0.661407,0.535445,0.515929,0.384615,0.624709,0.552268,0.500986,0.628205,0.854671,0.787879,0.766667,0.716578,0.371041,0.669683,0.611928,0.758494,0.743434,0.703857,0.424242,0.652681,0.531145,0.773333,0.584848,0.384615,0.707692,0.476852,0.599174,0.494755,0.58042,0.578283,0.825444,0.531065,0.577991,0.695266,0.668803,0.694444
905,907,f98419539,5,71,2014-09-11,2,3,13,-1.860675,1.0,907.0,0.802104,0.263198,0.637141,0.400855,0.695804,0.572978,0.585799,0.539886,0.83564,0.361854,0.377451,0.163102,0.542986,0.345023,0.449346,0.775023,0.522222,0.782369,0.456876,0.463869,0.51431,0.773333,0.486364,0.485897,0.589744,0.383333,0.855372,0.363636,0.562937,0.609848,0.781065,0.511834,0.606838,0.739645,0.577991,0.694444
906,908,fade343c8,8,78,2015-11-23,2,3,8,-1.868184,1.0,908.0,0.690335,0.427602,0.434343,0.455556,0.545455,0.639053,0.543393,0.58547,0.780277,0.631907,0.609804,0.469251,0.49095,0.464932,0.522059,0.752984,0.675758,0.636364,0.498834,0.441725,0.41835,0.762222,0.540909,0.478205,0.64359,0.512963,0.710744,0.51049,0.43007,0.435606,0.674556,0.454142,0.457265,0.636095,0.639957,0.625
907,909,fbc8a48a0,14,61,2017-03-20,2,3,9,-2.432351,0.0,909.0,0.741617,0.365008,0.717949,0.347863,0.722611,0.731755,0.522682,0.670228,0.678201,0.328877,0.603922,0.390374,0.354072,0.552036,0.409314,0.947658,0.617172,0.913223,0.688811,0.390443,0.302189,0.726667,0.589394,0.369231,0.520513,0.240741,0.859504,0.702797,0.512238,0.395202,0.840237,0.560651,0.719017,0.633136,0.588675,0.759259


In [6]:
classification = False
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "prmdiag", "Apoe", "IDs"]
target = "MEM_score"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [7]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
721,0.973914,-0.500490,0.690935,-0.851425,0.935651,-0.303436,-0.616518,0.996720,0.509876,-2.387953,-0.838634,-3.445535,-1.863701,-1.286178,-0.670299,-0.779123,-1.458162,-0.310851,-1.709077,-1.579579,-2.325631,-2.105735,-1.272134,-1.604870,-0.762529,0.214552,-1.016539,-0.486837,-0.517390,-2.199560,-2.520507,-0.412815,-0.458529,-1.034280,0.290805,1.190396,1.410052,0.965614,-0.660994,1.391845
335,0.281476,-0.174161,0.690935,0.819703,-0.701619,-0.527507,-1.713662,0.554782,-0.610574,-0.357812,1.434803,0.254674,-1.618042,0.693115,0.367284,0.512660,0.197101,-0.412706,-0.521696,0.960929,0.029824,-0.087567,-1.225184,-1.906351,-2.024973,-0.544471,0.802709,0.461626,-0.193724,-0.524046,0.454010,-1.649447,-0.177161,-0.938090,1.295154,-0.001445,0.159313,1.616143,0.641529,-0.032704
590,0.19615,-0.337326,-1.041167,0.151252,-1.510623,-0.247418,-0.014164,0.996720,-0.286588,-1.198317,0.425675,0.203163,-0.451163,0.872001,-0.717822,0.289269,0.462761,-1.709043,-0.711396,0.806290,0.392202,-1.240806,1.516729,-1.639325,-1.147574,0.448097,0.069369,0.820504,-0.334448,-1.028502,-0.183387,-0.428875,0.262476,-0.692269,1.449669,-0.581684,0.062169,1.494168,0.679467,1.010628
7,-1.815235,0.641659,0.690935,-0.851425,-1.106121,0.907948,0.272671,-0.165770,0.536875,0.793032,0.611262,-0.981591,-1.331440,0.854690,0.866274,1.163407,0.554720,1.226235,0.026325,-0.673833,1.039305,-0.979955,-0.004469,1.375486,0.215866,1.061153,0.153985,-0.858532,-0.010782,-0.307850,-1.351947,0.133231,0.983481,-0.104438,0.908866,-0.315088,-1.589292,-1.148603,-0.787452,-2.781483
567,0.676812,-0.174161,0.690935,-0.517199,0.993437,1.328081,1.233569,1.169652,0.118394,0.301660,0.437274,0.872806,0.838546,-0.426369,0.644500,1.901568,-0.037905,1.772548,1.361250,0.077274,-0.815724,-1.227077,-0.258002,-0.261126,-0.579474,-0.281732,-0.522944,0.576980,-0.024854,0.854200,-0.448969,0.068990,1.001067,0.184134,-0.211369,1.096303,0.620751,0.559034,0.135695,0.428770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,0.87983,-0.337326,-1.041167,0.151252,-1.144645,-1.304753,-0.050019,-0.703781,-0.246090,-1.612103,-1.708572,-2.046152,0.142513,-0.928405,0.288079,-0.108950,-0.988149,0.976227,0.321414,0.563285,0.935768,1.216691,-0.201661,1.995676,1.762360,-0.223346,0.266807,-0.063874,0.622479,0.683045,-0.342736,-0.043431,-0.124405,-0.072374,-0.520400,-0.048492,-1.091425,-1.351893,0.072466,-0.514242
793,-0.39345,0.967987,-1.041167,-0.517199,-0.220069,-1.577840,-0.136069,-0.809462,0.266887,-1.599172,-0.606651,-0.698280,0.633830,1.189381,0.842512,0.415533,0.769291,1.161418,0.194948,0.717924,0.072964,0.585155,1.328927,1.987062,2.393583,0.681642,0.703990,-0.307398,0.735058,0.647012,0.347777,1.369863,1.247264,1.263606,1.449669,0.547429,0.147170,0.193112,1.944052,1.652678
11,1.320656,-1.153146,0.690935,-0.851425,1.340153,2.084321,1.563429,3.062301,0.212889,1.530089,0.924439,0.529399,0.040155,0.029504,0.129669,0.580648,0.994080,1.355868,1.923324,-0.872655,-0.194505,0.516510,-0.342513,0.341837,0.102246,0.418904,0.591169,0.794870,0.791348,0.043467,0.294661,-0.862499,-0.370602,-0.991529,0.136290,0.876753,0.827184,-0.213469,0.780634,1.993768
317,0.627507,-0.174161,-1.041167,1.488154,0.261481,-0.527507,-0.673885,0.343420,0.104894,-1.224178,1.539195,1.353576,0.388171,1.022035,1.666242,0.930303,-0.477265,0.337318,1.276939,1.137660,1.436195,1.944330,0.399306,1.685581,1.680301,1.440665,1.521947,-0.089508,1.115014,1.791046,1.941268,0.647156,1.423119,1.423924,-1.833779,-0.832597,-0.775705,0.762324,1.273822,0.809987


# Train Model

In [8]:
from src.models.pipeline_elastic_net import model_elastic_net
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, classification = False, 
                          n_alphas_linreg = 10, 
                          l1_ratios_linreg= np.linspace(0.01,  1, 11).tolist(),
                          cv_linreg = 5)               

In [10]:
predictions = model.predict(Xtest)
mean_squared_error(ytest, predictions)

0.6478621394705101

In [11]:
r2_score(ytest, predictions)

0.33830267112539736

In [12]:
model.l1_ratio_

0.01

In [13]:
model.alpha_

0.1832473458910573

# Save Model

In [14]:
import pickle

In [15]:
filename ='pretrained_elnet_regr_agg_zero.sav'
pickle.dump(model, open(filename, 'wb'))

# Visualisation Feature Importance and Coeffs (evtl Ridge)

In [ ]:
# load model
model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/Innolab/pretrained_elnet_regr_agg_zero.sav", 'rb'))